# MCMC2.5: Dense and Sparse Matrices

This section is not necessarily for Marcov chain Monte Carlo itself, but the use of Sparse matrices is important to speed up quantum Monte Carlo simulations.

## Dense matrix

To use dense matrices, LinearAlgebra provides wrapper to BLAS/LAPACK.

In [1]:
using LinearAlgebra

I recommend Intel MKL instead of OpenBLAS. You can check whether MKL is used by the following command:

In [2]:
BLAS.vendor()

:mkl

1D Array is called Vector.

In [3]:
Array{Int64, 1} == Vector{Int64}

true

2D Array is called Matrix, which I later call dense matrix.

In [4]:
Array{Float64, 2} == Matrix{Float64}

true

Matrix and Array of Array are different.

In [5]:
matrix = [1 2
                 3 4]
array =[[1, 2], [3, 4]]
matrix == array

false

Julia's support on 3-rank tensors is limited, but still we can define and use them.

In [6]:
tensor = zeros(Float64, 2, 2, 2)

2×2×2 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 2] =
 0.0  0.0
 0.0  0.0

Be careful, matrices are stored columnwise.

In [7]:
mat = ones(10000, 10000)
mat[:, 1]
mat[1, :]
@time mat[:, 1];
@time mat[1, :];

  0.000024 seconds (7 allocations: 78.375 KiB)
  0.000412 seconds (7 allocations: 78.375 KiB)


Thus, vertical vectors are more important.

In [8]:
[1.0 2.0; 3.0 4.0] * [5.0; 6.0]

2-element Array{Float64,1}:
 17.0
 39.0

## BLAS/LAPACK

Dense matrices are not memory-efficient, but support BLAS/LAPACK. The most important BLAS operation in quantum Monte Carlo is rank-1 update (or other finite-rank updates).

In [9]:
A = rand(Float64, 1000, 1000)
Ainv = inv(A)

1000×1000 Array{Float64,2}:
  0.0431439    0.0888779   …  -0.232161     0.0334541  -0.10434  
 -0.235971    -0.401377        0.271148    -0.0722883  -0.195389 
 -0.0387278   -0.128991        0.0114505   -0.0752756  -0.100264 
 -0.0289467    0.121712        0.0573061    0.139086    0.0810516
 -0.212922    -0.0187286       0.233624     0.135971   -0.149635 
 -0.0580575   -0.176289    …   0.129684     0.0379991  -0.0129432
  0.142148     0.141984       -0.263332     0.0531205   0.0427841
  0.109876     0.0526541      -0.0992558    0.0143842   0.112669 
  0.136653     0.00726805     -0.178598    -0.0400401   0.0482636
 -0.126976    -0.137025        0.146843    -0.0466192  -0.0871268
  0.0941911    0.231223    …  -0.0192677    0.0992502   0.188856 
 -0.0315682   -0.144275        0.17446      0.0830541  -0.0187862
 -0.187044    -0.063006        0.0813068    0.0908393  -0.159311 
  ⋮                        ⋱                                     
 -0.0551023    0.129739        0.161075     0.06

Let's check the Sherman-Morrison formula!
$$\left( A+\vec{u} \vec{v}^T \right)^{-1} = A^{-1} - \frac{A^{-1} \vec{u} \vec{v}^TA^{-1}}{1 + \vec{v}^T A^{-1} \vec{u}}$$
Of course, the vectors are vertical. I define $B = A+\vec{u} \vec{v}^T$

In [10]:
u = rand(Float64, 1000)
v = rand(Float64, 1000)
B = copy(A)
BLAS.ger!(1.0, u, v, B)

1000×1000 Array{Float64,2}:
 0.832968  0.652178  0.724346  1.19486   …  0.334528  0.730802  0.928351
 1.3461    0.45736   0.662928  0.968384     0.905952  0.555805  0.772106
 0.249889  0.121346  0.439088  0.448305     0.254457  0.653032  0.805448
 0.820503  0.398808  0.436075  0.477687     0.677882  0.453538  0.764802
 0.740269  0.468934  0.632645  0.661285     0.874208  0.650122  0.982307
 0.343132  0.569241  0.648693  0.789917  …  0.251079  0.903195  0.706866
 1.55833   1.14814   0.659995  1.12276      1.25377   0.93118   0.467273
 0.128137  0.821006  0.674959  0.817371     0.1994    0.958172  0.126384
 0.963283  0.966691  0.981831  1.23097      0.33287   0.949716  1.14575 
 0.810313  1.36004   1.44107   1.53647      0.960035  0.39253   1.19298 
 1.1866    0.86983   0.953029  0.340716  …  0.176998  0.710034  1.26138 
 0.306116  0.604526  0.444104  0.938056     0.951188  0.475478  0.612429
 0.972492  0.578318  0.667652  0.819561     1.05324   0.497566  1.24237 
 ⋮                     

You may need to copy the matrix first because most BLAS operations are destructive. The rank-1 update is a BLAS-2 function, so it is faster than the BLAS-3 function, matrix inversion. That's why we use the Sherman-Morrison formula to calculate $B^{-1}$ when we already know $A^{-1}$.

In [11]:
Binv = copy(Ainv)
BLAS.ger!(-1.0 / (1.0 + v' * Ainv * u), Ainv * u, (v' * Ainv)', Binv)

1000×1000 Array{Float64,2}:
  0.0207023    0.013859     0.0422636  …   0.00714479  -0.133266 
 -0.114238     0.00555833   0.0145399      0.0704248   -0.0384843
  0.00567708   0.0194481   -0.0264304     -0.0232178   -0.0430294
 -0.04895      0.0548441   -0.0803093      0.115635     0.055269 
 -0.115136     0.308156     0.273267       0.25061     -0.0235963
 -0.022912    -0.0588034    0.0558867  …   0.0792017    0.0323565
  0.0822957   -0.058091    -0.0287768     -0.0170463   -0.0343602
  0.0368055   -0.191608    -0.0344167     -0.0712792    0.0184869
  0.0599814   -0.249034    -0.0483716     -0.129926    -0.0505605
 -0.0423401    0.1459       0.22952        0.0526034    0.0219626
  0.0452352    0.067571    -0.0780428  …   0.041857     0.125756 
 -0.027322    -0.130081    -0.0556808      0.0880321   -0.0133131
 -0.171071    -0.00960869  -0.0462855      0.109566    -0.138722 
  ⋮                                    ⋱                         
 -0.0670159    0.0899132   -0.0324655      0.050

Let's check the results.

In [12]:
norm(B * Binv - I)

7.424705419107949e-11

Iterating rank-1 updates accumulates some error, so sometimes you have to refresh the updated matrix "from scratch." (Be careful because sometimes it is not really from scratch.)

As for LAPACK, most Julia functions on linear algebra are just wrappers of LAPACK.

In [13]:
eigvals(A)

1000-element Array{Complex{Float64},1}:
    500.2148852665727 + 0.0im                
   -6.098333785458043 + 7.017296010747078im  
   -6.098333785458043 - 7.017296010747078im  
     -8.0972931651208 + 4.207552150579319im  
     -8.0972931651208 - 4.207552150579319im  
   -8.807360262124128 + 2.2185591531094784im 
   -8.807360262124128 - 2.2185591531094784im 
    -8.35892169465259 + 3.411774904994613im  
    -8.35892169465259 - 3.411774904994613im  
   -9.047722150615531 + 0.27918731766378635im
   -9.047722150615531 - 0.27918731766378635im
   -7.543040464081789 + 4.76467779877881im   
   -7.543040464081789 - 4.76467779877881im   
                      ⋮                      
  -1.1729019206274498 + 0.29718356345534697im
  -1.1729019206274498 - 0.29718356345534697im
  0.22307321051946205 + 0.7350647422279357im 
  0.22307321051946205 - 0.7350647422279357im 
   -0.817132240547356 + 0.2126900153686736im 
   -0.817132240547356 - 0.2126900153686736im 
  0.13457895726107866 + 0.32441861872762

eigvals is just a wrapper for LAPACK.geevx!, so you can directely call LAPACK.geevx! instead if you wish.

## Sparse matrix

If your program is intensively using sparse matrices, you should use python instead because Julia only supports CSC matrix. Julia's native support for sparse matrices is not strong, so I do not recommend to write a code using multiple types of sparse matrices in Julia.

In [14]:
using SparseArrays

Let's solve a tight-binding model on the 2D square lattice in a poor man's way, i.e. in the real space.

In [15]:
const L = 30
iter1D = 1 : L
nnbondx = zip(Iterators.product(iter1D, iter1D), Iterators.product((mod1(i + 1, L) for i in iter1D), iter1D))
nnbondy = zip(Iterators.product(iter1D, iter1D), Iterators.product(iter1D, (mod1(i + 1, L) for i in iter1D)))
collect(nnbondx), collect(nnbondy)

(Tuple{Tuple{Int64,Int64},Tuple{Int64,Int64}}[((1, 1), (2, 1)) ((1, 2), (2, 2)) … ((1, 29), (2, 29)) ((1, 30), (2, 30)); ((2, 1), (3, 1)) ((2, 2), (3, 2)) … ((2, 29), (3, 29)) ((2, 30), (3, 30)); … ; ((29, 1), (30, 1)) ((29, 2), (30, 2)) … ((29, 29), (30, 29)) ((29, 30), (30, 30)); ((30, 1), (1, 1)) ((30, 2), (1, 2)) … ((30, 29), (1, 29)) ((30, 30), (1, 30))], Tuple{Tuple{Int64,Int64},Tuple{Int64,Int64}}[((1, 1), (1, 2)) ((1, 2), (1, 3)) … ((1, 29), (1, 30)) ((1, 30), (1, 1)); ((2, 1), (2, 2)) ((2, 2), (2, 3)) … ((2, 29), (2, 30)) ((2, 30), (2, 1)); … ; ((29, 1), (29, 2)) ((29, 2), (29, 3)) … ((29, 29), (29, 30)) ((29, 30), (29, 1)); ((30, 1), (30, 2)) ((30, 2), (30, 3)) … ((30, 29), (30, 30)) ((30, 30), (30, 1))])

These iterators will generate the 2D square lattice.

In [16]:
xytoz(nn::Tuple{Tuple{Int64, Int64}, Tuple{Int64, Int64}}) = (nn[1][2] - 1) * L + nn[1][1], (nn[2][2] - 1) * L + nn[2][1]
nnx = Base.Generator(xytoz, nnbondx)
nny = Base.Generator(xytoz, nnbondy)

Base.Generator{Base.Iterators.Zip2{Base.Iterators.ProductIterator{Tuple{UnitRange{Int64},UnitRange{Int64}}},Base.Iterators.ProductIterator{Tuple{UnitRange{Int64},Base.Generator{UnitRange{Int64},getfield(Main, Symbol("##5#6"))}}}},typeof(xytoz)}(xytoz, Base.Iterators.Zip2{Base.Iterators.ProductIterator{Tuple{UnitRange{Int64},UnitRange{Int64}}},Base.Iterators.ProductIterator{Tuple{UnitRange{Int64},Base.Generator{UnitRange{Int64},getfield(Main, Symbol("##5#6"))}}}}(Base.Iterators.ProductIterator{Tuple{UnitRange{Int64},UnitRange{Int64}}}((1:30, 1:30)), Base.Iterators.ProductIterator{Tuple{UnitRange{Int64},Base.Generator{UnitRange{Int64},getfield(Main, Symbol("##5#6"))}}}((1:30, Base.Generator{UnitRange{Int64},getfield(Main, Symbol("##5#6"))}(getfield(Main, Symbol("##5#6"))(), 1:30)))))

Base.Generator(f, iter) is same as (f(x) for x in iter).

In [17]:
N = L ^ 2
H = spzeros(Float64, N, N)
for (i, j) in Iterators.flatten((nnx, nny))
    H[i, j] = -1.0
    H[j, i] = -1.0
end
H

900×900 SparseMatrixCSC{Float64,Int64} with 3600 stored entries:
  [2  ,   1]  =  -1.0
  [30 ,   1]  =  -1.0
  [31 ,   1]  =  -1.0
  [871,   1]  =  -1.0
  [1  ,   2]  =  -1.0
  [3  ,   2]  =  -1.0
  [32 ,   2]  =  -1.0
  [872,   2]  =  -1.0
  [2  ,   3]  =  -1.0
  [4  ,   3]  =  -1.0
  [33 ,   3]  =  -1.0
  [873,   3]  =  -1.0
  ⋮
  [28 , 898]  =  -1.0
  [868, 898]  =  -1.0
  [897, 898]  =  -1.0
  [899, 898]  =  -1.0
  [29 , 899]  =  -1.0
  [869, 899]  =  -1.0
  [898, 899]  =  -1.0
  [900, 899]  =  -1.0
  [30 , 900]  =  -1.0
  [870, 900]  =  -1.0
  [871, 900]  =  -1.0
  [899, 900]  =  -1.0

Note that you can rewrite this code by zip, but zip is not efficient here.

Most of the operations for sparse matrices are similar to the ones for dense matrices. However, sparse arrays are more memory-efficient when the components of the matrix is almost zero. Especially, if the matrix is sparse enough, it significantly reduces the matrix muliplication cost from $O(N^3)$ to $O(N)$.

In [19]:
Hdense = Array(H)
H * H
Hdense * Hdense
@time H * H;
@time Hdense * Hdense;

  0.000187 seconds (18 allocations: 268.500 KiB)
  0.022350 seconds (6 allocations: 6.180 MiB)


eigvals does not support sparse matrices, so the calculation of the whole eigenvalues still costs $O(N^3)$. I will discuss this problem later in MCMC5.0.

In [20]:
eigvals(Hdense)

900-element Array{Float64,1}:
 -4.000000000000004 
 -3.9562952014676074
 -3.956295201467607 
 -3.9562952014676056
 -3.9562952014676043
 -3.9125904029352245
 -3.912590402935224 
 -3.9125904029352214
 -3.912590402935216 
 -3.827090915285207 
 -3.827090915285198 
 -3.8270909152851953
 -3.827090915285193 
  ⋮                 
  3.8270909152851997
  3.8270909152852073
  3.82709091528521  
  3.9125904029352205
  3.9125904029352228
  3.912590402935223 
  3.912590402935234 
  3.9562952014676016
  3.956295201467605 
  3.956295201467612 
  3.9562952014676163
  3.999999999999999 

For the square (or cubic, etc.) lattice, you can directly begin from a dense matrix. Here's a smart implementation.

In [22]:
H4d = zeros(Float64, L, L, L, L)
for ((i, j), (k, l)) in Iterators.flatten((nnbondx, nnbondy))
    H4d[i, j, k, l] = -1.0
    H4d[k, l, i, j] = -1.0
end
H2d = reshape(H4d, N, N)
eigvals(H2d)

900-element Array{Float64,1}:
 -4.000000000000004 
 -3.9562952014676074
 -3.956295201467607 
 -3.9562952014676056
 -3.9562952014676043
 -3.9125904029352245
 -3.912590402935224 
 -3.9125904029352214
 -3.912590402935216 
 -3.827090915285207 
 -3.827090915285198 
 -3.8270909152851953
 -3.827090915285193 
  ⋮                 
  3.8270909152851997
  3.8270909152852073
  3.82709091528521  
  3.9125904029352205
  3.9125904029352228
  3.912590402935223 
  3.912590402935234 
  3.9562952014676016
  3.956295201467605 
  3.956295201467612 
  3.9562952014676163
  3.999999999999999 

## Block checkerboard decomposition/approximation

It is sometimes very useful to approximate a dense matrix by a product of sparse matrices. In the physical models like tight-binding models, block checkerboard docomposition will be a good approximation.

~ under construction ~

## Iterative solvers

Iterative solvers, especially conjugate gradient methods are important for hybrid Monte Carlo simulations for lattice gauge theories.

~ under construction ~